# Regression with an Insurance Dataset

## Keras - Attempt 1

## Attempt 1

In [2]:
# Reproducibility
import os, random, time, json
import numpy as np
import pandas as pd
random.seed(42)
np.random.seed(42)
import tensorflow as tf
from sklearn.utils import all_estimators
# scikit-learn has no global seed; we set seeds in train_test_split
tf.random.set_seed(42)

# Imports
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Load Data
df_train = pd.read_csv('playground-series-s4e12/train.csv')
df_test = pd.read_csv('playground-series-s4e12/test.csv')
# Identify id and target columns
id_col = 'id'
target_columns = ['Premium Amount']

# Combine and encode target
df = df_train.copy()
y_values = df[target_columns].astype(float).values
# Use log1p if all non-negative
if np.all(y_values >= 0):
    y_enc = np.log1p(y_values)
else:
    y_enc = y_values

# Prepare features
drop_cols = [id_col] + target_columns
X = df.drop(columns=drop_cols, errors='ignore')
# Testing set
X_test = df_test.drop(columns=drop_cols, errors='ignore')
test_ids = df_test[id_col]

# Feature engineering: drop cols with all missing
non_null_cols = X.columns[X.notna().any()].tolist()
X = X[non_null_cols]
X_test = X_test[non_null_cols]
# Identify categorical vs numeric
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Drop high-cardinality categorical cols
high_card = [c for c in categorical_cols if X[c].nunique() > 50]
for c in high_card:
    X.drop(columns=c, inplace=True)
    X_test.drop(columns=c, inplace=True)
categorical_cols = [c for c in categorical_cols if c not in high_card]
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Preprocessing pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

# Fit and transform
df_train_proc = preprocessor.fit_transform(X)
df_test_proc = preprocessor.transform(X_test)
n_samples, n_features = df_train_proc.shape

# Build model (small dataset branch)
from tensorflow.keras import layers, models, optimizers
hidden_units = [min(n_features*2, 128), min(n_features, 64)]
model = models.Sequential()
for units in hidden_units:
    model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='linear'))

# Compile
model.compile(
    optimizer=optimizers.Adam(),
    loss='mean_squared_error',
    metrics=[RootMeanSquaredError(name='rmse'), MeanAbsoluteError(name='mae')]
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Training
time_start = time.time()
history = model.fit(
    df_train_proc, y_enc,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)
time_duration = time.time() - time_start

# Save results
results = {
    'training_loss': history.history['loss'][-1],
    'validation_loss': history.history['val_loss'][-1],
    'training_rmse': history.history['rmse'][-1],
    'validation_rmse': history.history['val_rmse'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(df_test_proc)
# Reverse log
final = np.expm1(np.clip(raw_preds, a_min=None, a_max=20))
# Ensure 2D
if final.ndim == 1:
    final = final.reshape(-1,1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

2025-07-09 21:42:42.139689: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100


15000/15000 - 24s - 2ms/step - loss: 1.7048 - mae: 0.9848 - rmse: 1.3057 - val_loss: 1.1586 - val_mae: 0.7775 - val_rmse: 1.0764
Epoch 2/100


15000/15000 - 22s - 1ms/step - loss: 1.1823 - mae: 0.7873 - rmse: 1.0873 - val_loss: 1.1495 - val_mae: 0.7651 - val_rmse: 1.0721
Epoch 3/100


15000/15000 - 23s - 2ms/step - loss: 1.1640 - mae: 0.7752 - rmse: 1.0789 - val_loss: 1.1468 - val_mae: 0.7655 - val_rmse: 1.0709
Epoch 4/100
15000/15000 - 24s - 2ms/step - loss: 1.1606 - mae: 0.7728 - rmse: 1.0773 - val_loss: 1.1469 - val_mae: 0.7657 - val_rmse: 1.0709
Epoch 5/100


...

15000/15000 - 22s - 1ms/step - loss: 1.1557 - mae: 0.7700 - rmse: 1.0750 - val_loss: 1.1428 - val_mae: 0.7637 - val_rmse: 1.0690
Epoch 29/100
15000/15000 - 22s - 1ms/step - loss: 1.1555 - mae: 0.7700 - rmse: 1.0749 - val_loss: 1.1430 - val_mae: 0.7635 - val_rmse: 1.0691
Epoch 30/100
15000/15000 - 23s - 2ms/step - loss: 1.1555 - mae: 0.7700 - rmse: 1.0750 - val_loss: 1.1430 - val_mae: 0.7625 - val_rmse: 1.0691
Epoch 31/100


15000/15000 - 22s - 1ms/step - loss: 1.1555 - mae: 0.7699 - rmse: 1.0749 - val_loss: 1.1425 - val_mae: 0.7638 - val_rmse: 1.0689
Epoch 32/100


15000/15000 - 22s - 1ms/step - loss: 1.1556 - mae: 0.7699 - rmse: 1.0750 - val_loss: 1.1424 - val_mae: 0.7630 - val_rmse: 1.0688
Epoch 33/100
15000/15000 - 22s - 1ms/step - loss: 1.1554 - mae: 0.7699 - rmse: 1.0749 - val_loss: 1.1425 - val_mae: 0.7636 - val_rmse: 1.0689
Epoch 34/100
15000/15000 - 21s - 1ms/step - loss: 1.1551 - mae: 0.7697 - rmse: 1.0748 - val_loss: 1.1427 - val_mae: 0.7632 - val_rmse: 1.0690
Epoch 35/100


15000/15000 - 19s - 1ms/step - loss: 1.1553 - mae: 0.7699 - rmse: 1.0748 - val_loss: 1.1423 - val_mae: 0.7635 - val_rmse: 1.0688
Epoch 36/100


15000/15000 - 19s - 1ms/step - loss: 1.1557 - mae: 0.7699 - rmse: 1.0750 - val_loss: 1.1421 - val_mae: 0.7614 - val_rmse: 1.0687
Epoch 37/100
15000/15000 - 19s - 1ms/step - loss: 1.1553 - mae: 0.7698 - rmse: 1.0749 - val_loss: 1.1423 - val_mae: 0.7640 - val_rmse: 1.0688
Epoch 38/100
15000/15000 - 20s - 1ms/step - loss: 1.1550 - mae: 0.7697 - rmse: 1.0747 - val_loss: 1.1430 - val_mae: 0.7636 - val_rmse: 1.0691
Epoch 39/100
15000/15000 - 19s - 1ms/step - loss: 1.1550 - mae: 0.7697 - rmse: 1.0747 - val_loss: 1.1435 - val_mae: 0.7637 - val_rmse: 1.0693
Epoch 40/100
15000/15000 - 20s - 1ms/step - loss: 1.1551 - mae: 0.7698 - rmse: 1.0748 - val_loss: 1.1440 - val_mae: 0.7633 - val_rmse: 1.0696
Epoch 41/100
15000/15000 - 21s - 1ms/step - loss: 1.1551 - mae: 0.7697 - rmse: 1.0748 - val_loss: 1.1426 - val_mae: 0.7642 - val_rmse: 1.0689
Epoch 42/100
15000/15000 - 21s - 1ms/step - loss: 1.1554 - mae: 0.7698 - rmse: 1.0749 - val_loss: 1.1432 - val_mae: 0.7628 - val_rmse: 1.0692
Epoch 43/100
15000/

# Keras Tuner - 1 Attempt

# Attempt 1

In [ ]:
# Reproducibility
import os, random, time, json
import numpy as np
import pandas as pd
random.seed(42)
np.random.seed(42)
import tensorflow as tf
from sklearn.utils import all_estimators
# scikit-learn has no global seed; we set seeds in train_test_split
tf.random.set_seed(42)

# Imports
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Load Data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
# Identify id and target columns
id_col = 'id'
target_columns = ['Premium Amount']

# Combine and encode target
df = df_train.copy()
y_values = df[target_columns].astype(float).values
# Use log1p if all non-negative
if np.all(y_values >= 0):
    y_enc = np.log1p(y_values)
else:
    y_enc = y_values

# Prepare features
drop_cols = [id_col] + target_columns
X = df.drop(columns=drop_cols, errors='ignore')
# Testing set
X_test = df_test.drop(columns=drop_cols, errors='ignore')
test_ids = df_test[id_col]

# Feature engineering: drop cols with all missing
non_null_cols = X.columns[X.notna().any()].tolist()
X = X[non_null_cols]
X_test = X_test[non_null_cols]
# Identify categorical vs numeric
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Drop high-cardinality categorical cols
high_card = [c for c in categorical_cols if X[c].nunique() > 50]
for c in high_card:
    X.drop(columns=c, inplace=True)
    X_test.drop(columns=c, inplace=True)
categorical_cols = [c for c in categorical_cols if c not in high_card]
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Preprocessing pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

# Fit and transform
df_train_proc = preprocessor.fit_transform(X)
df_test_proc = preprocessor.transform(X_test)
n_samples, n_features = df_train_proc.shape

# Build model (Keras-Tuner snippet)
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Define early stopping and checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Preserve input dim
n_features = df_train_proc.shape[1]

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-05, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        x = Dense(1, activation='linear')(x)
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='mean_squared_error', metrics=[RootMeanSquaredError(name='rmse'), MeanAbsoluteError(name='mae')])
        return model

# Tuner setup
bs = 64  # batch size
ep = 100  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=False ,
    project_name='bayesian_tuner'
)

if y_enc is not None:
    tuner.search(
        df_train_proc, y_enc,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        df_train_proc, y_enc,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(
    tuner.get_best_hyperparameters(1)[0]
)

# Retrain model with original callbacks and data
start_time = time.time()  # Start timing
if y_enc is not None:
    history = model.fit(
        df_train_proc, y_enc,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        df_train_proc, y_enc,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
end_time = time.time()  # End timing
duration = end_time - start_time  # Calculate duration

# Save results
results = {
    'training_loss': history.history['loss'][-1],
    'validation_loss': history.history['val_loss'][-1],
    'training_rmse': history.history['rmse'][-1],
    'validation_rmse': history.history['val_rmse'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(df_test_proc)
# Reverse log
final = np.expm1(np.clip(raw_preds, a_min=None, a_max=20))
# Ensure 2D
if final.ndim == 1:
    final = final.reshape(-1,1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 21m 06s]
val_loss: 1.1446022987365723

Best val_loss So Far: 1.1321594715118408
Total elapsed time: 04h 26m 48s
Epoch 1/100


2025-07-15 19:10:00.840071: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 496 bytes spill stores, 496 bytes spill loads

2025-07-15 19:10:00.889840: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 240 bytes spill stores, 240 bytes spill loads

2025-07-15 19:10:00.943026: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 152 bytes spill stores, 152 bytes spill loads

2025-07-15 19:10:01.026897: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 240 bytes spill stores, 240 bytes spill loads

2025-07-15 19:10:01.084641: I external/l

15000/15000 - 82s - 5ms/step - loss: 1.2421 - mae: 0.8094 - rmse: 1.1145 - val_loss: 1.1509 - val_mae: 0.7695 - val_rmse: 1.0728
Epoch 2/100


15000/15000 - 68s - 5ms/step - loss: 1.1792 - mae: 0.7825 - rmse: 1.0859 - val_loss: 1.1480 - val_mae: 0.7706 - val_rmse: 1.0714
Epoch 3/100


15000/15000 - 70s - 5ms/step - loss: 1.1715 - mae: 0.7782 - rmse: 1.0824 - val_loss: 1.1471 - val_mae: 0.7699 - val_rmse: 1.0710
Epoch 4/100


...

15000/15000 - 71s - 5ms/step - loss: 1.1546 - mae: 0.7703 - rmse: 1.0745 - val_loss: 1.1399 - val_mae: 0.7652 - val_rmse: 1.0677
Epoch 17/100


15000/15000 - 65s - 4ms/step - loss: 1.1535 - mae: 0.7697 - rmse: 1.0740 - val_loss: 1.1385 - val_mae: 0.7643 - val_rmse: 1.0670
Epoch 18/100
15000/15000 - 72s - 5ms/step - loss: 1.1533 - mae: 0.7696 - rmse: 1.0739 - val_loss: 1.1394 - val_mae: 0.7625 - val_rmse: 1.0674
Epoch 19/100
15000/15000 - 69s - 5ms/step - loss: 1.1537 - mae: 0.7695 - rmse: 1.0741 - val_loss: 1.1392 - val_mae: 0.7639 - val_rmse: 1.0673
Epoch 20/100
15000/15000 - 70s - 5ms/step - loss: 1.1531 - mae: 0.7694 - rmse: 1.0738 - val_loss: 1.1409 - val_mae: 0.7630 - val_rmse: 1.0681
Epoch 21/100
15000/15000 - 71s - 5ms/step - loss: 1.1533 - mae: 0.7695 - rmse: 1.0739 - val_loss: 1.1416 - val_mae: 0.7659 - val_rmse: 1.0684
Epoch 22/100
15000/15000 - 75s - 5ms/step - loss: 1.1532 - mae: 0.7693 - rmse: 1.0739 - val_loss: 1.1429 - val_mae: 0.7657 - val_rmse: 1.0691
Epoch 23/100
15000/15000 - 78s - 5ms/step - loss: 1.1536 - mae: 0.7698 - rmse: 1.0741 - val_loss: 1.1414 - val_mae: 0.7647 - val_rmse: 1.0684
Epoch 24/100
15000/

15000/15000 - 65s - 4ms/step - loss: 1.1506 - mae: 0.7683 - rmse: 1.0727 - val_loss: 1.1375 - val_mae: 0.7611 - val_rmse: 1.0665
Epoch 28/100
15000/15000 - 70s - 5ms/step - loss: 1.1506 - mae: 0.7686 - rmse: 1.0727 - val_loss: 1.1391 - val_mae: 0.7621 - val_rmse: 1.0673
Epoch 29/100
15000/15000 - 75s - 5ms/step - loss: 1.1493 - mae: 0.7678 - rmse: 1.0721 - val_loss: 1.1403 - val_mae: 0.7633 - val_rmse: 1.0678
Epoch 30/100
15000/15000 - 76s - 5ms/step - loss: 1.1496 - mae: 0.7679 - rmse: 1.0722 - val_loss: 1.1402 - val_mae: 0.7629 - val_rmse: 1.0678
Epoch 31/100
15000/15000 - 75s - 5ms/step - loss: 1.1489 - mae: 0.7675 - rmse: 1.0719 - val_loss: 1.1406 - val_mae: 0.7628 - val_rmse: 1.0680
Epoch 32/100
15000/15000 - 71s - 5ms/step - loss: 1.1508 - mae: 0.7686 - rmse: 1.0728 - val_loss: 1.1427 - val_mae: 0.7650 - val_rmse: 1.0690
Epoch 33/100
15000/15000 - 72s - 5ms/step - loss: 1.1511 - mae: 0.7688 - rmse: 1.0729 - val_loss: 1.1400 - val_mae: 0.7624 - val_rmse: 1.0677
Epoch 34/100
15000/

2025-07-15 19:53:39.895328: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 152 bytes spill stores, 152 bytes spill loads

2025-07-15 19:53:39.921430: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 364 bytes spill stores, 364 bytes spill loads

2025-07-15 19:53:39.937520: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 56 bytes spill stores, 56 bytes spill loads

2025-07-15 19:53:39.988357: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 64 bytes spill stores, 64 bytes spill loads

2025-07-15 19:53:39.994059: I external/local

25000/25000 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step


In [1]:
print(duration)

2688.784352
